Alpha (α), Beta (ß), Gamma (Γ), Delta (δ), Epsilon (ε), Theta (Θ), Omega (Ω), Pi (π), Mu (µ), Sigma (Σ, σ), Tau (τ), Phi (Φ, φ). Dérivée: ∂

# Modèle Black Litterman - Explications et implémentation avec yahoo finance

Inputs: Expected return + covariance
Outputs: Optimal weights

A mix between what you think and what market thinks

Process: use the implied returns from the market and your expectations and come up with optimal weigts

Helper Functions:

In [2]:
import numpy as np 
import pandas as pd 
import yfinance as yf
from numpy.linalg import inv 
from pandas_datareader import data
from pandas.util.testing import assert_frame_equal

# function vect columns 
def as_colvec(x):
        if(x.ndim ==2 ):
            return x 
        else : 
            return np.expand_dims(x,axis=1)
        
as_colvec(np.arange(4))

array([[0],
       [1],
       [2],
       [3]])

Master Equation: π = δΣw

If the investor doesn't want to set confidence level on his views, then Ω is a matrix proportional to the covariance matrix of the prior: 

Ω = diag(P(τΣ)P.T)

In [4]:
def implied_returns(delta, sigma, w, rf=0.02):
#delta = risk aversion coefficient (scalar)
#sigma = Var-Cov matirx 
#w = weights as series 

#Obtain the implied expected returns by reverse engineering the weights

    ir = delta*sigma.dot(w).squeeze()+rf #Remove single-dimensional entries from the shape of an array.
    ir.name = 'Implied Returns'
    return ir 


def proportional_prior(sigma,tau,p): 
    #tau = scalar
    #p = KXN matrix DF, a matrix representing prior uncertainties
    helit_omega = p.dot(tau*sigma).dot(p.T)
    return pd.DataFrame(np.diag(np.diag(helit_omega.values)), index=p.index, columns=p.index)


def bl(w_prior, sigma_prior, p, q, omega=None, delta=2.5, tau=.02): 
    if omega is None : 
        omega = proportional_prior(sigma_prior, tau, p)
    #how many asset do we have?
    N=w_prior.shape[0]
    #how many views ?
    K=q.shape[0]
    #First, reverse engineer the weigts to get pi
    pi = implied_returns(delta, sigma_prior, w_prior)
    #Adjust (scale) Sigma by the uncertainty factor
    sigma_prior_scaled = tau * sigma_prior
    
    mu_bl = pi + sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+omega).dot(q-p.dot(pi).values))
    sigma_bl = sigma_prior + sigma_prior_scaled - sigma_prior_scaled.dot(p.T).dot(inv(p.dot(sigma_prior_scaled).dot(p.T)+ omega)).dot(p).dot(sigma_prior_scaled)
        
    return(mu_bl, sigma_bl)

def inverse(d): 
    #Invert the dataframe by inverting the underlying matrix
    return pd.DataFrame(inv(d.values), index = d.columns, columns = d.index)

def w_msr(sigma, mu, scale = True):
    w= inverse(sigma).dot(mu)
    if scale : 
        w = w/sum(w) 
    return (w)

## Implementation avec yahoo finance

In [17]:
tickers = ['AAPL', 'AMZN']
startdate = '2019-01-01'
enddate = '2020-01-01'
prices = yf.download(tickers, start=startdate, end=enddate)['Adj Close']
returns = prices.pct_change().dropna()
covdf=pd.DataFrame(returns.cov())*252
mkcap = data.get_quote_yahoo(tickers)['marketCap']
mkcap = mkcap/mkcap.sum(axis=0) #Rapport des market cap

#s=pd.DataFrame([[46.0, 1.06], [1.06, 5.33]], index=tickers, columns=tickers)*10E-4
#pi = implied_returns(delta = 2.5, sigma=s , w=pd.Series([.44, .56], index= tickers))
pi = implied_returns(delta = 2.5, sigma=covdf , w=mkcap)

print('-----------------------Implied Prior Returns from Black Litterman Model')
print(pi)
print('-----------------------')

mu_exp = pd.Series([.10,.05], index= tickers ) 

#Absolute views for optimal portfolio (markowitz)
#we perform markowitz optim with these views as expected returns in the model. Ie, 10% for Apple and 5% for Amazon.

MarkoWeights = np.round(w_msr(covdf,mu_exp)*100,2)

print('-----------------------Weights from Markowitz Optimisation with our absolute views as Expected Returns')
print(MarkoWeights)
print('-----------------------')

q=pd.Series({'AAPL':0.10, 'AMZN': 0.05}) #Absolute views for BL and matching matrix (p and q)
p=pd.DataFrame([
    {'AAPL':1, 'AMZN': 0},
    {'AAPL':0, 'AMZN': 1}
    ])
bl_mu , bl_sigma = bl(w_prior=mkcap, sigma_prior=covdf, p=p, q=q, tau=0.01)

#The posterior returns returned by the procedure are clearly weighted between that of:
# -the equilibrium implied expected returns
# -and that of the investor.

print('-----------------------Posterior Expected Returns (mu) from BL Model')
print(bl_mu)
print('-----------------------')

print('-----------------------Posterior Covariance (sigma) from BL Model')
print(bl_sigma)
print('-----------------------')

#Much more reasonable weights
BLWeights = w_msr(bl_sigma,bl_mu)
print('-----------------------Weights from Optim with BL Model Inputs (both mu and sigma considering abs. views)')
print(BLWeights)
print('-----------------------')

[*********************100%***********************]  2 of 2 completed
-----------------------Implied Prior Returns from Black Litterman Model
AAPL    0.152430
AMZN    0.128168
Name: Implied Returns, dtype: float64
-----------------------
-----------------------Weights from Markowitz Optimisation with our absolute views as Expected Returns
AAPL    103.3
AMZN     -3.3
dtype: float64
-----------------------
-----------------------Posterior Expected Returns (mu) from BL Model
AAPL    0.114324
AMZN    0.085386
dtype: float64
-----------------------
-----------------------Posterior Covariance (sigma) from BL Model
          AAPL      AMZN
AAPL  0.068631  0.035369
AMZN  0.035369  0.052723
-----------------------
-----------------------Weights from Optim with BL Model Inputs (both mu and sigma considering abs. views)
AAPL    0.623438
AMZN    0.376562
dtype: float64
-----------------------


In [18]:
q=pd.Series([0.02]) #Relative views
p = pd.DataFrame([ #Intel outperforms pfizer by 2%
    {'AAPL':+1, 'AMZN': -1}
])
bl_mu , bl_sigma = bl(w_prior=mkcap, sigma_prior=covdf, p=p, q=q)

print('-----------------------Posterior Expected Returns (mu) from BL Model')
print(bl_mu)
print('-----------------------')

#Black Litterman implied Mu
#again, weights are blended between cap-weight implied weights AND
#the investor view

diff1 = pi[0]-pi[1] 
print('-----------------------Difference between Implied Prior Returns:')
print(diff1)
print('-----------------------')
#outperformance of intel in the implied returns

diff2 = bl_mu[0]-bl_mu[1]
print('-----------------------Difference between BL Expected Returns (mu):')
print(diff2)
print('-----------------------')

views = w_msr(bl_sigma,bl_mu) #New weights including relative views.
print('-----------------------New weights from Optim relative views:')
print(views)
print('-----------------------')

-----------------------Posterior Expected Returns (mu) from BL Model
AAPL    0.151029
AMZN    0.128898
dtype: float64
-----------------------
-----------------------Difference between Implied Prior Returns:
0.024261704873173973
-----------------------
-----------------------Difference between BL Expected Returns (mu):
0.022130852436586967
-----------------------
-----------------------New weights from Optim relative views:
AAPL    0.494272
AMZN    0.505728
dtype: float64
-----------------------


In [19]:
bl_sigma

,AAPL,AMZN
AAPL,0.069470,0.036091
AMZN,0.036091,0.053477


## The He litterman paper 

In [7]:
countries = ['AU', 'CA', 'FR', 'DE', 'JP', 'UK','US']

#correlation matrix
rho =pd.DataFrame([
    [1.000,0.488,0.478,0.515,0.439,0.512,0.491],
    [0.488,1.000,0.664,0.655,0.310,0.608,0.779],
    [0.478,0.664,1.000,0.861,0.355,0.783,0.668],
    [0.515,0.655,0.861,1.000,0.354,0.777,0.653],
    [0.439,0.310,0.355,0.354,1.000,0.405,0.306],
    [0.512,0.608,0.783,0.777,0.405,1.000,0.652],
    [0.491,0.779,0.668,0.653,0.306,0.652,1.000]
], index= countries, columns=countries)
vols= pd.DataFrame([0.160,0.203,0.248,0.271,0.210,0.200,0.187], index= countries, columns=["Vols"])
w_eq = pd.DataFrame([0.016,0.022,0.052,0.055,0.116,0.124,0.615], index= countries, columns=["CapWeight"])
sigma_prior = vols.dot(vols.T)*rho #Covariance matrix from correlation
pi = implied_returns(delta=2.5, sigma=sigma_prior, w=w_eq)
(pi*100).round(1)
# Views germany vs Europe
#Germany will outperform other European Equities by 5%
q=pd.Series([0.05])
#one single view, array of zeros and overwrite the specific view
p=pd.DataFrame([0.]*len(countries), index=countries).T
#relative market cap
w_fr = w_eq.loc["FR"]/(w_eq.loc["FR"]+w_eq.loc["UK"])
w_uk =  w_eq.loc["UK"]/(w_eq.loc["FR"]+w_eq.loc["UK"])
p.iloc[0]['DE'] = 1.
p.iloc[0]['FR'] = -w_fr
p.iloc[0]['UK'] = -w_uk
#P matrix is telling you how the view is affecting the asset:
(p*100).round(1)

tau = 0.05
delta = 2.5

bl_mu , bl_sigma = bl(w_eq, sigma_prior, p, q, tau=tau)
(bl_mu*100).round(1)

bl_mu.sum()


def w_star(delta, sigma, mu): 
    return( inverse(sigma).dot(mu))/delta

w_star = w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
(w_star*100).round(1)

w_star.sum()

#Spot the difference between posterior and prior weights
w_eq = w_msr(delta*sigma_prior, pi, scale=False)
w_eq
np.round(w_star-w_eq/(1+tau),3)*100

(w_star*100).round(1)

AU    20.9
CA     6.3
FR    -4.4
DE    26.3
JP    18.6
UK    -0.3
US    68.7
dtype: float64